In [ ]:
"""
Notebook section: 1.3.10 Feature evaluation and refinement
Team 102D · AB Data Challenge — Iteration 2

This script follows after 1.3.9 (Feature Engineering Implementation).
It loads results/iteration_2/features_v2.csv and performs:

1.3.10.1 Identification of relevant and non‑redundant features
  • Basic filters: missingness, zero/low variance
  • Univariate signals: point-biserial correlation, MI, single‑feature AUC
  • Redundancy pruning: correlation-based clustering/greedy removal
  • Model-based importance (LightGBM → fallback to RF / L1-LogReg)
  • Optional VIF for multi-collinearity diagnostics (slow; disabled by default)

1.3.10.2 Summary of selected features for modeling
  • Persist: selected_features.txt, feature_importances.csv, redundancy_pairs.csv
  • Report: feature_selection_report.md (human-readable summary)

Safe to run even if LightGBM isn’t installed.
"""

# === Imports & setup ===
import os
import json
import math
import warnings
import numpy as np
import pandas as pd

from typing import List, Dict, Tuple

from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

# Optional: LightGBM if available
try:
    import lightgbm as lgb  # type: ignore
    _HAS_LGB = True
except Exception:
    _HAS_LGB = False

warnings.filterwarnings("ignore")

# === Paths ===
PROJECT_ROOT = "."
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
RESULTS_DIR = os.path.join(PROJECT_ROOT, "results", "iteration_2")
os.makedirs(RESULTS_DIR, exist_ok=True)

FEATS_CSV = os.path.join(RESULTS_DIR, "features_v2.csv")

print("Looking for features:", FEATS_CSV)
if not os.path.exists(FEATS_CSV):
    raise SystemExit("features_v2.csv not found. Run 1.3.9 first.")

# === Load ===
df = pd.read_csv(FEATS_CSV, low_memory=False)
print("Loaded features:", df.shape)

# === Columns & label ===
ID_CANDIDATES = ["num_serie_contador", "polissa_id"]
TIME_COLS = ["datetime", "date", "data_inici", "data_fi", "year", "month", "dayofweek", "hour"]
LABEL = "y_anom" if "y_anom" in df.columns else None

id_col = None
for c in ID_CANDIDATES:
    if c in df.columns:
        id_col = c
        break

# === Helper: choose numeric feature set (excludes IDs, time, labels) ===
EXCLUDE = set((ID_CANDIDATES + TIME_COLS + ["codi_anomalia", "meter_mean", "meter_std"]))
if LABEL:
    EXCLUDE.add(LABEL)

num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c]) and c not in EXCLUDE]
if not num_cols:
    raise SystemExit("No numeric feature columns found after exclusions.")

print(f"Numeric feature candidates: {len(num_cols)}")

# === Basic diagnostics ===
miss_ratio = df[num_cols].isna().mean().sort_values(ascending=False)
zero_var = [c for c in num_cols if df[c].std(skipna=True) == 0]

print(f"Zero-variance features: {len(zero_var)}")

# Persist basic stats
miss_ratio.to_csv(os.path.join(RESULTS_DIR, "fs_missingness.csv"))
pd.Series(zero_var, name="zero_var_features").to_csv(os.path.join(RESULTS_DIR, "fs_zero_variance.csv"), index=False)

# Remove obviously bad ones now
usable = [c for c in num_cols if c not in zero_var and miss_ratio.get(c, 0.0) < 0.99]
print(f"Usable after basic filters: {len(usable)}")

# === Unsupervised low-variance threshold (optional mild) ===
VAR_THRESH = 1e-10
low_var = [c for c in usable if df[c].var(skipna=True) <= VAR_THRESH]
usable = [c for c in usable if c not in low_var]
if low_var:
    pd.Series(low_var, name="low_var_features").to_csv(os.path.join(RESULTS_DIR, "fs_low_variance.csv"), index=False)
print(f"Usable after low-var filter: {len(usable)}")

# === Supervised signals (only if label present) ===
results = {}
if LABEL is not None and df[LABEL].notna().any():
    y = df[LABEL].astype(int)

    # Point-biserial (Pearson) correlation with label
    def safe_corr(x: pd.Series, y: pd.Series) -> float:
        s = pd.concat([x, y], axis=1).dropna()
        if len(s) < 50 or s.iloc[:,0].std(ddof=1) == 0 or s.iloc[:,1].std(ddof=1) == 0:
            return np.nan
        return float(s.iloc[:,0].corr(s.iloc[:,1]))

    corr_with_y = {c: safe_corr(df[c], y) for c in usable}

    # Mutual information
    X_mi = df[usable].copy()
    imputer = SimpleImputer(strategy="median")
    X_mi = imputer.fit_transform(X_mi)
    mi_vals = mutual_info_classif(X_mi, y, discrete_features=False, random_state=42)
    mi_series = pd.Series(mi_vals, index=usable)

    # Single-feature AUC (using raw values as scores)
    auc_vals = {}
    for c in usable:
        s = pd.concat([df[c], y], axis=1).dropna()
        if s[LABEL].nunique() < 2:
            auc_vals[c] = np.nan
            continue
        try:
            auc_vals[c] = roc_auc_score(s[LABEL], s[c])
        except Exception:
            auc_vals[c] = np.nan
    auc_series = pd.Series(auc_vals)

    # Persist univariate metrics
    uni_df = pd.DataFrame({
        "feature": usable,
        "corr_y": [corr_with_y.get(c) for c in usable],
        "mutual_info": [mi_series.get(c, np.nan) for c in usable],
        "auc_uni": [auc_series.get(c, np.nan) for c in usable],
        "missing_ratio": [miss_ratio.get(c, np.nan) for c in usable],
        "variance": [df[c].var(skipna=True) for c in usable],
    }).sort_values(["mutual_info"], ascending=False)
    uni_df.to_csv(os.path.join(RESULTS_DIR, "fs_univariate_metrics.csv"), index=False)

    # === Redundancy pruning (correlation-based) ===
    # Use pairwise Pearson on imputed data
    X_imp = pd.DataFrame(imputer.fit_transform(df[usable]), columns=usable)
    corr_mat = X_imp.corr().abs()
    np.fill_diagonal(corr_mat.values, 0.0)

    REDUNDANCY_THRESH = 0.95
    to_drop = set()
    pairs = []

    # Greedy: for each high-corr pair, drop the one with lower MI (fallback to lower variance)
    cols_sorted = list(usable)
    for i, a in enumerate(cols_sorted):
        if a in to_drop:
            continue
        high = corr_mat.index[(corr_mat.loc[a] >= REDUNDANCY_THRESH)].tolist()
        for b in high:
            if b == a or b in to_drop:
                continue
            mia = float(mi_series.get(a, 0.0))
            mib = float(mi_series.get(b, 0.0))
            vara = float(df[a].var(skipna=True))
            varb = float(df[b].var(skipna=True))
            drop_b = mib < mia if not (math.isnan(mia) or math.isnan(mib)) else (varb < vara)
            drop = b if drop_b else a
            keep = a if drop_b else b
            to_drop.add(drop)
            pairs.append({"a": a, "b": b, "corr": float(corr_mat.loc[a, b]), "kept": keep, "dropped": drop})

    red_pairs = pd.DataFrame(pairs)
    if not red_pairs.empty:
        red_pairs.sort_values("corr", ascending=False).to_csv(os.path.join(RESULTS_DIR, "fs_redundant_pairs.csv"), index=False)

    pruned = [c for c in usable if c not in to_drop]
    print(f"After redundancy pruning @ {REDUNDANCY_THRESH}: {len(pruned)} (dropped {len(to_drop)})")

    # === Model-based importance (cross-validated) ===
    groups = df[id_col] if id_col is not None else None
    n_splits = 5
    splitter = GroupKFold(n_splits=n_splits) if groups is not None else StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    feat_importances = []
    perm_rows = []

    # Choose model
    if _HAS_LGB:
        model_name = "LightGBM"
    else:
        model_name = "RandomForest"

    print(f"Model for importance: {model_name}")

    for fold, (tr, va) in enumerate(splitter.split(df, y, groups) if groups is not None else splitter.split(df, y), 1):
        X_tr = df.iloc[tr][pruned]
        X_va = df.iloc[va][pruned]
        y_tr = y.iloc[tr]
        y_va = y.iloc[va]

        X_tr_imp = imputer.fit_transform(X_tr)
        X_va_imp = imputer.transform(X_va)

        if _HAS_LGB:
            clf = lgb.LGBMClassifier(
                n_estimators=600,
                learning_rate=0.03,
                max_depth=-1,
                subsample=0.9,
                colsample_bytree=0.9,
                reg_alpha=0.0,
                reg_lambda=0.0,
                random_state=42,
                n_jobs=-1,
                class_weight="balanced"
            )
            clf.fit(X_tr_imp, y_tr, eval_set=[(X_va_imp, y_va)], eval_metric="auc", verbose=False)
            imp = clf.booster_.feature_importance(importance_type="gain")
        else:
            clf = RandomForestClassifier(
                n_estimators=600,
                max_depth=None,
                min_samples_leaf=1,
                n_jobs=-1,
                class_weight="balanced_subsample",
                random_state=42,
            )
            clf.fit(X_tr_imp, y_tr)
            imp = clf.feature_importances_

        auc_val = roc_auc_score(y_va, clf.predict_proba(X_va_imp)[:,1])
        print(f"  Fold {fold}/{n_splits} AUC = {auc_val:.4f}")

        feat_importances.append(pd.Series(imp, index=pruned, name=f"fold{fold}"))

        # Permutation importance on validation fold (fast-ish)
        perm = permutation_importance(clf, X_va_imp, y_va, n_repeats=5, random_state=42, n_jobs=-1, scoring="roc_auc")
        perm_rows.append(pd.Series(perm.importances_mean, index=pruned, name=f"perm_fold{fold}"))

    imp_df = pd.concat(feat_importances, axis=1)
    imp_df["mean_importance"] = imp_df.mean(axis=1)
    imp_df["std_importance"] = imp_df.std(axis=1)
    imp_df = imp_df.sort_values("mean_importance", ascending=False)
    imp_df.to_csv(os.path.join(RESULTS_DIR, "fs_model_importances.csv"))

    perm_df = pd.concat(perm_rows, axis=1)
    perm_df["mean_perm"] = perm_df.mean(axis=1)
    perm_df["std_perm"] = perm_df.std(axis=1)
    perm_df = perm_df.sort_values("mean_perm", ascending=False)
    perm_df.to_csv(os.path.join(RESULTS_DIR, "fs_permutation_importances.csv"))

    # === Combine signals into a single score ===
    # Normalize each metric to [0,1] via rank, then average
    def rank01(s: pd.Series) -> pd.Series:
        s = s.fillna(-np.inf)
        r = s.rank(method="average", ascending=False)
        return (r - r.min()) / (r.max() - r.min() + 1e-9)

    combo = pd.DataFrame(index=pruned)
    combo["mi_r"] = rank01(mi_series.reindex(pruned))
    combo["auc_r"] = rank01(auc_series.reindex(pruned))
    combo["corr_r"] = rank01(pd.Series(corr_with_y).reindex(pruned).abs())
    combo["imp_r"] = rank01(imp_df["mean_importance"].reindex(pruned))
    combo["perm_r"] = rank01(perm_df["mean_perm"].reindex(pruned))
    combo["missing_penalty"] = 1.0 - miss_ratio.reindex(pruned).fillna(0.0).clip(0, 1)

    combo["score"] = (
        0.30 * combo["imp_r"] +
        0.20 * combo["perm_r"] +
        0.20 * combo["mi_r"] +
        0.15 * combo["auc_r"] +
        0.10 * combo["corr_r"] +
        0.05 * combo["missing_penalty"]
    )

    combo = combo.sort_values("score", ascending=False)
    combo.to_csv(os.path.join(RESULTS_DIR, "fs_combined_scores.csv"))

    # === Gatekeeping: must-pass thresholds (very mild) ===
    GATE_MI = 0.0005
    GATE_AUC = 0.53
    GATE_CORR = 0.02

    gates = (
        (mi_series.reindex(pruned).fillna(0.0) >= GATE_MI) |
        (auc_series.reindex(pruned).fillna(0.0) >= GATE_AUC) |
        (pd.Series(corr_with_y).reindex(pruned).abs().fillna(0.0) >= GATE_CORR)
    )

    candidates = combo[gates].copy()

    # Cap final selection size for first pass (tuneable)
    TOP_K = min(60, len(candidates))
    selected = candidates.head(TOP_K).index.tolist()

    # Persist selections & summaries
    with open(os.path.join(RESULTS_DIR, "selected_features_v1.txt"), "w") as f:
        for c in selected:
            f.write(c + "\n")

    summary = {
        "n_total_numeric": int(len(num_cols)),
        "n_after_basic_filters": int(len(usable)),
        "n_after_redundancy": int(len(pruned)),
        "n_selected": int(len(selected)),
        "model": model_name,
        "redundancy_threshold": REDUNDANCY_THRESH,
        "gates": {"mi": GATE_MI, "auc": GATE_AUC, "abs_corr": GATE_CORR},
        "id_col": id_col,
        "label": LABEL,
    }
    with open(os.path.join(RESULTS_DIR, "fs_summary.json"), "w") as f:
        json.dump(summary, f, indent=2)

    # Human-readable report
    lines = []
    lines.append("# 1.3.10 — Feature Evaluation & Refinement\n")
    lines.append("## Inputs\n")
    lines.append(f"- Source: results/iteration_2/features_v2.csv ({df.shape[0]} rows, {df.shape[1]} cols)\n")
    lines.append(f"- Label: {LABEL} | ID: {id_col} | Model: {model_name}\n")
    lines.append("\n## Filtering\n")
    lines.append(f"- Zero-variance removed: {len(zero_var)}\n")
    lines.append(f"- Low-variance removed (≤ {VAR_THRESH}): {len(low_var)}\n")
    lines.append(f"- After redundancy pruning @ {REDUNDANCY_THRESH}: {len(pruned)}\n")
    lines.append("\n## Univariate signals (top 15 by MI)\n")
    lines.append(uni_df.head(15).to_markdown(index=False))
    lines.append("\n\n## Model importances (top 20)\n")
    lines.append(imp_df.head(20)[["mean_importance","std_importance"]].to_markdown())
    lines.append("\n\n## Permutation importances (top 20)\n")
    lines.append(perm_df.head(20)[["mean_perm","std_perm"]].to_markdown())
    lines.append("\n\n## Selected features (first pass)\n")
    lines.append("```")
    lines.extend(selected)
    lines.append("`````\n")

    report_path = os.path.join(RESULTS_DIR, "feature_selection_report.md")
    with open(report_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    print("[saved] selected_features_v1.txt, fs_* CSVs, and feature_selection_report.md")

else:
    # No labels available → produce an unsupervised shortlist based on stability & missingness
    print("[info] Label not available — running unsupervised selection.")
    usable = [c for c in usable if miss_ratio.get(c, 1.0) <= 0.3]

    # Redundancy pruning via Pearson
    imputer = SimpleImputer(strategy="median")
    X_imp = pd.DataFrame(imputer.fit_transform(df[usable]), columns=usable)
    corr_mat = X_imp.corr().abs()
    np.fill_diagonal(corr_mat.values, 0.0)

    REDUNDANCY_THRESH = 0.95
    to_drop = set()
    for a in usable:
        if a in to_drop:
            continue
        for b in corr_mat.index[(corr_mat.loc[a] >= REDUNDANCY_THRESH)].tolist():
            if b == a or b in to_drop:
                continue
            # drop lower variance
            drop = b if df[b].var(skipna=True) < df[a].var(skipna=True) else a
            to_drop.add(drop)
    pruned = [c for c in usable if c not in to_drop]

    selected = pruned[:50]
    with open(os.path.join(RESULTS_DIR, "selected_features_v1.txt"), "w") as f:
        for c in selected:
            f.write(c + "\n")

    report_path = os.path.join(RESULTS_DIR, "feature_selection_report.md")
    with open(report_path, "w", encoding="utf-8") as f:
        f.write("\n".join([
            "# 1.3.10 — Feature Evaluation & Refinement (Unsupervised)",
            f"- Source: {FEATS_CSV} ({df.shape[0]} rows)",
            f"- Selected (top 50 after redundancy): {len(selected)}",
            "",
            "First-pass unsupervised shortlist written to selected_features_v1.txt",
        ]))

    print("[saved] selected_features_v1.txt and feature_selection_report.md (unsupervised)")

# === Optional: quick visuals (each in its own Matplotlib figure) ===
import matplotlib.pyplot as plt

# Missingness top-20
mr = miss_ratio.head(20)
plt.figure()
mr.plot(kind="bar")
plt.title("Top-20 Missingness Ratio by Feature")
plt.xlabel("Feature")
plt.ylabel("Missingness")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# If label, show combined score distribution
if LABEL is not None and os.path.exists(os.path.join(RESULTS_DIR, "fs_combined_scores.csv")):
    combo = pd.read_csv(os.path.join(RESULTS_DIR, "fs_combined_scores.csv"))
    plt.figure()
    pd.Series(combo["score"]).hist(bins=40)
    plt.title("Distribution of Combined Feature Scores")
    plt.xlabel("Score")
    plt.ylabel("Count")
    plt.show()

print("Done.")
